<h1>Importaciones</h1>

In [1]:
import pandas as pd
import openpyxl
import xlrd
import pyarrow as pa
import pyarrow.parquet as pq

import warnings
warnings.filterwarnings(action= 'ignore')

%load_ext autoreload
%autoreload 2
import resources as r




<h1>Fichero Informacion Contable</h1>

In [34]:
# Cargar el archivo Excel
file_path = '../data/raw/info_contable_ypf_2020_1.xls'  # path del fichero.xlsx
workbook = xlrd.open_workbook(file_path) # instancio la carga del fichero en una variable

# Crear un diccionario para almacenar los DataFrames de las tablas dinámicas
pivot_tables = {} # instancio un diccionario para guardar la etiqueta de la hoja y su data en formato dataframe

# Iterar sobre cada hoja del archivo
for idx, sheet_name in enumerate(workbook.sheet_names(), start=1): # itero sobre cada hoja tomando como referencia las etiquetas de cada una con el metodo sheetnames

    sheet = workbook.sheet_by_name(sheet_name)  # Instancia la hoja
    
    # Identificar la tabla dinámica (puedes ajustar este criterio según tu necesidad)
    # Aquí asumimos que la tabla dinámica empieza en la fila 2 y columna B (cambiar según sea necesario)
    pivot_start_row = 1  # Las filas en xlrd están indexadas desde 0
    pivot_start_col = 1  # Las columnas en xlrd están indexadas desde 0
    
    # Extraer la tabla dinámica
    pivot_data = []
    for row_idx in range(pivot_start_row, sheet.nrows):
        row = sheet.row_values(row_idx, start_colx=pivot_start_col)
        pivot_data.append(row)
    
    # Convertir a DataFrame de pandas
    pivot_df = pd.DataFrame(pivot_data[1:], columns = pivot_data[0])
    
    # # Almacenar la tabla dinámica en el diccionario
    pivot_tables[sheet_name] = pivot_df

# Mostrar las tablas dinámicas extraídas
for sheet_name, pivot_table in pivot_tables.items(): # iteramos sobre el nombre de la hoja (clave) y sobre el dataframe con los datos (valor) del diccionario
    año = 2020 # inicializamos una variable que contenga el año al que refieren los datos
    print(f'Tabla dinámica en la hoja: {sheet_name}')
    # print(pivot_table)
    # print('\n')
    df = pd.DataFrame(pivot_tables[sheet_name]) # instanciamos los datos de cada hoja en un dataframe
    df_transposed = df.T # transponemos los datos

    df_transposed.columns = pivot_table.iloc[:, 0] 
    df_transposed = df_transposed.drop(df_transposed.index[0])
    df_transposed['Año'] = año

    if '' in df_transposed.columns:
        df_transposed.drop(columns= '', inplace= True)


    archivo= f'../data/raw/info_contable/{sheet_name}.parquet'
    try:
        pq.write_table(pa.Table.from_pandas(df_transposed), archivo)
    except ValueError:
        df_transposed.columns = list(r.make_unique(df_transposed.columns))
        pq.write_table(pa.Table.from_pandas(df_transposed), archivo)
    
    print(df_transposed.head())
    print(df_transposed.index)
    print('-'*100)
    print()
# print(f'Nombre de dataframe: {sheet_name}')
# pivot_df.head()


Tabla dinámica en la hoja: P&L
        (Amounts expressed in million of Argentine Pesos)   Revenues  \
1Q 2019                                                     130907.0   
4Q 2019                                                     206910.0   
1Q 2020                                                     174670.0   

            Costs Gross Profit Selling Expenses Administration Expenses  \
1Q 2019 -104754.0      26153.0          -9820.0                 -4768.0   
4Q 2019 -187044.0      19866.0         -16963.0                 -8124.0   
1Q 2020 -145914.0      28756.0         -13876.0                 -6749.0   

        Exploration Expenses Other operating results, net Operating Income  \
1Q 2019              -1521.0                        587.0          10631.0   
4Q 2019              -2348.0                       -617.0          -8186.0   
1Q 2020               -716.0                       7383.0          14798.0   

        Income (loss) of interest in companies and joint ventures 

<h1>Fichero Inversiones</h1>

In [13]:
## Extraemos la tabla estatica de inversiones por empresa por año.

# r.conversion_de_tablas_dinamicas('../data/raw/TD Inversiones_2024.xlsx', 3, 12, 'A', 2024)
# r.conversion_de_tablas_dinamicas('../data/raw/TD Inversiones_2023.xlsx', 3, 12, 'A', 2023)
# r.conversion_de_tablas_dinamicas('../data/raw/TD Inversiones_2022.xlsx', 3, 12, 'A', 2022)
# r.conversion_de_tablas_dinamicas('../data/raw/TD Inversiones_2021.xlsx', 3, 14, 'A', 2021)
# r.conversion_de_tablas_dinamicas('../data/raw/td_2057_2020.xlsx', 3, 12, 'A', 2020)

<h1>Fichero Reservas</h1>

In [27]:
file_path = '../data/raw/reservas al 31-12-2022.xlsx'  # path del fichero.xlsx
workbook = openpyxl.load_workbook(file_path, data_only=True) # instancio la carga del fichero en una variable

# Iterar sobre cada hoja del archivo
for idx, sheet_name in enumerate(workbook.sheetnames, start=1): # itero sobre cada hoja tomando como referencia las etiquetas de cada una con el metodo sheetnames
    
    if idx != 2:  # Solo procesar la segunda hoja
        continue

    sheet = workbook[sheet_name] # instancio la hoja
    
    # Identificar la tabla dinámica (puedes ajustar este criterio según tu necesidad)
    # Aquí asumimos que la tabla dinámica empieza en la fila 12 y columna A (cambiar según sea necesario)
    pivot_start_row = 7
    pivot_start_col = 'A'
    
    # Extraer la tabla dinámica
    pivot_data = []
    for row in sheet.iter_rows(min_row=pivot_start_row, min_col=openpyxl.utils.cell.column_index_from_string(pivot_start_col), values_only=True):
        pivot_data.append(row)

    columnas = list(pivot_data[0])
    new_columns = []
    num = 1
    for idx, columna in enumerate(columnas):
        if idx >= 21:
            if idx == 21:
                temp = 'reserv_comp_pet_' + columna
            elif idx == 22:
                temp = 'reserv_comp_gas_' + columna
            elif idx == 23:
                temp = 'reserv_prob_pet_' + columna
            elif idx == 24:
                temp = 'reserv_prob_gas_' + columna
            elif idx == 25:
                temp = 'reserv_pos_pet_' + columna
            elif idx == 26:
                temp = 'reserv_pos_gas_' + columna
            elif idx == 27:
                temp = 'recur_cont_pet_' + columna
            elif idx == 28:
                temp = 'recur_cont_gas_' + columna
            new_columns.append(temp)
        elif idx > 4 and idx < 21:
            temp = str(num) + columna
            num += 1
            new_columns.append(temp)
        else:
            new_columns.append(columna)

    print(columnas)
    print(len(columnas))
    print(new_columns)
    print(new_columns[5:21])
    
    # Convertir a DataFrame de pandas
    pivot_df = pd.DataFrame(pivot_data[1:], columns = new_columns)
    pivot_df.drop(columns= pivot_df.columns[5:21], inplace= True)
    pivot_df.drop(index= [1256,1257], inplace= True)

    archivo= f'../data/raw/reservas_{sheet_name}_2023.parquet'
    pq.write_table(pa.Table.from_pandas(pivot_df), archivo)
        

['OPERADOR', 'CUENCA', 'PROVINCIA', 'CONCESIÓN O PERMISO', 'YACIMIENTO', '(Mm3)', '(MMm3)', '(Mm3)', '(MMm3)', '(Mm3)', '(MMm3)', '(Mm3)', '(MMm3)', '(Mm3)', '(MMm3)', '(Mm3)', '(MMm3)', '(Mm3)', '(MMm3)', '(Mm3)', '(MMm3)', '(Mm3)', '(MMm3)', '(Mm3)', '(MMm3)', '(Mm3)', '(MMm3)', '(Mm3)', '(MMm3)']
29
['OPERADOR', 'CUENCA', 'PROVINCIA', 'CONCESIÓN O PERMISO', 'YACIMIENTO', '1(Mm3)', '2(MMm3)', '3(Mm3)', '4(MMm3)', '5(Mm3)', '6(MMm3)', '7(Mm3)', '8(MMm3)', '9(Mm3)', '10(MMm3)', '11(Mm3)', '12(MMm3)', '13(Mm3)', '14(MMm3)', '15(Mm3)', '16(MMm3)', 'reserv_comp_pet_(Mm3)', 'reserv_comp_gas_(MMm3)', 'reserv_prob_pet_(Mm3)', 'reserv_prob_gas_(MMm3)', 'reserv_pos_pet_(Mm3)', 'reserv_pos_gas_(MMm3)', 'recur_cont_pet_(Mm3)', 'recur_cont_gas_(MMm3)']
['1(Mm3)', '2(MMm3)', '3(Mm3)', '4(MMm3)', '5(Mm3)', '6(MMm3)', '7(Mm3)', '8(MMm3)', '9(Mm3)', '10(MMm3)', '11(Mm3)', '12(MMm3)', '13(Mm3)', '14(MMm3)', '15(Mm3)', '16(MMm3)']


<h1>Ficheros World</h1>

In [14]:
r.extraccion_de_tablas('../data/raw/world-oil-domestic-consumption-statistics.xlsx',1,6,'B',66,'AL','world_oil_domestic_consumption')

Nombre de dataframe: Yearbook world_oil_domestic_consumption


,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,...,2016,2017,2018,2019,2020,2021,2022,2023,2022 - 2023 (%),2000 - 2023 (%/year)
World,3043.741867,3069.187186,3087.126637,3094.659766,3120.551918,3179.060993,3263.624464,3380.005935,3433.639666,3501.026212,...,4185.520965,4261.567139,4279.847861,4302.964561,3906.362046,4125.900613,4238.887789,4341.656373,2.424423,0.894809
OECD,1809.645174,1821.628400,1857.684113,1872.818439,1923.526550,1938.964355,1987.553193,2046.321643,2067.011761,2092.307852,...,1846.398343,1849.201798,1840.615587,1833.638476,1647.707862,1736.390769,1761.285843,1750.129255,-0.633434,-0.741383
G7,1388.415985,1383.605398,1404.570017,1410.907692,1437.864422,1441.352272,1474.524790,1520.560681,1545.268947,1564.034256,...,1295.196293,1292.605507,1294.893154,1283.115333,1142.230957,1209.588853,1226.670956,1222.737425,-0.320667,-1.005466
BRICS,438.180259,458.694569,448.449735,445.919190,412.603042,427.668618,446.096686,469.965537,479.340148,500.221374,...,989.538550,1035.355897,1070.905679,1101.218022,1087.955661,1157.379574,1191.242783,1268.916776,6.520417,3.844834
Europe,649.537072,656.107992,653.690981,648.273656,651.586698,663.333181,681.269342,681.144097,690.326849,680.869987,...,573.793957,583.951787,572.653904,570.165090,520.627810,544.698777,546.309919,534.576845,-2.147695,-0.959687
European Union,524.257072,532.169992,531.645981,524.023656,529.643698,539.598181,551.666342,553.787097,565.434849,556.517099,...,447.104056,454.749761,446.944190,446.347753,407.285635,426.315525,426.906919,414.732215,-2.851840,-1.175154
Belgium,17.596700,19.187800,19.641100,19.167800,21.557100,21.454300,22.874100,22.890200,22.922700,22.564100,...,20.909522,20.227052,20.371082,19.378545,17.617904,18.871734,17.740842,17.520138,-1.244044,-1.095860
Czechia,8.746000,7.297000,7.819000,7.400000,7.508000,7.794000,8.047000,7.765000,8.114000,7.999000,...,7.774000,9.146000,9.208000,9.272000,8.368000,9.187000,9.284000,9.000392,-3.054805,0.658097
Denmark,7.395000,7.716000,7.497000,7.610000,7.955000,8.392000,9.324000,8.962000,8.592000,8.556000,...,5.717927,5.703709,5.711280,5.560163,5.248605,5.349797,5.453007,5.429340,-0.434020,-1.798005
France,80.382000,85.928000,85.288000,82.516000,81.843000,84.092000,85.248000,85.402000,89.012000,88.085000,...,69.595293,70.241979,67.391694,66.943269,59.450760,63.855164,62.219591,61.583451,-1.022411,-1.459780


In [15]:
r.extraccion_de_tablas('../data/raw/world-production-statistics.xlsx',1,6,'B',66,'AL','world_production')

Nombre de dataframe: Yearbook world_production


,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,...,2016,2017,2018,2019,2020,2021,2022,2023,2022 - 2023 (%),2000 - 2023 (%/year)
World,3193.320561,3179.009174,3196.753465,3219.257661,3258.272984,3304.865128,3390.656128,3489.667631,3585.863119,3565.893348,...,4424.740376,4418.424144,4519.569681,4513.254638,4204.670854,4250.560486,4463.235350,4506.154712,0.961620,0.907782
OECD,921.192000,941.621000,957.725000,965.983000,1002.783568,1020.930589,1050.194999,1067.273000,1069.106000,1044.668000,...,1135.693599,1171.691294,1279.016946,1355.993213,1310.691296,1323.403553,1366.735490,1440.273894,5.380588,1.339654
G7,614.936000,614.908000,617.213000,616.906000,639.103000,644.329000,646.273000,648.549000,648.193000,632.092000,...,830.975000,882.966673,1003.026119,1089.225590,1034.581487,1044.641806,1092.125178,1156.660061,5.909110,2.702960
BRICS,729.221000,675.574000,602.433000,560.052000,531.741000,526.246000,539.684000,546.475000,550.502000,557.541000,...,928.294200,923.773100,927.101800,942.367727,907.825956,914.363083,940.493948,955.630399,1.609415,2.143011
Europe,220.890000,231.290900,247.183600,260.110600,301.844700,314.946900,332.485800,330.975200,327.839900,333.831300,...,170.773327,168.869480,167.624551,163.621256,172.772462,165.419220,155.201697,157.690732,1.603743,-3.277501
European Union,40.347000,39.193900,39.061600,38.932600,40.555700,39.775900,39.530800,40.517200,39.752900,40.914300,...,27.607285,27.795309,27.368156,25.772527,24.450742,23.046937,21.005116,20.987819,-0.082346,-3.210813
Belgium,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.033500,0.053400,0.050600,0.039000,0.036500,0.041845,14.643008,NaN
Czechia,0.220000,0.221000,0.237000,0.257000,0.270000,0.276000,0.251000,0.463000,0.433000,0.386000,...,0.187000,0.213000,0.199000,0.163000,0.095000,0.087000,0.078000,0.070000,-10.256410,-7.143926
Denmark,5.994000,6.993000,7.756000,8.265000,9.118000,9.170000,10.122000,11.145000,11.432000,14.465000,...,6.924374,6.736967,5.665787,5.017234,3.520207,3.236752,3.184860,2.922014,-8.252984,-7.551004
France,3.470000,3.798000,3.564000,3.458000,3.387000,3.027000,2.648000,2.266000,2.092000,2.001000,...,0.992000,0.964000,0.912907,0.964135,0.818663,0.864083,0.811973,0.842182,3.720406,-3.491045


<h1>Fichero Ratios YPF</h1>

## <h1>Fichero Ratios Empresas</h1>

In [3]:
r.extraccion_de_tablas('../data/raw/Ratios_empresas.xlsx',1,4,'O',22,'S', 'ratios')

Nombre de dataframe: REPSOL ratios


,2023,2022,2021,2020
RDT,0.371629,0.423840,1.620237,0.509360
RDC,0.499503,0.520497,0.506295,0.589795
RDLP,0.430882,NaN,NaN,NaN
None,NaN,NaN,NaN,NaN
Actividad,2023.000000,2022.000000,2021.000000,2020.000000
Rotación de Inv.,6.156490,5.517143,8.199311,7.603312
Rotación de A.T.,0.105788,0.139817,0.833667,0.118638
Rotación de C. por c.,0.000000,0.000000,0.000000,0.000000
None,NaN,NaN,NaN,NaN
Rentabilidad,2023.000000,2022.000000,2021.000000,2020.000000


In [4]:
r.extraccion_de_tablas('../data/raw/Ratios_empresas.xlsx',2,5,'M',23,'O', 'ratios')

Nombre de dataframe: REPSOL ratios


,2024,2023
RDT,0.371457,0.439192
RDC,0.590981,0.783143
RDLP,0.305592,0.480528
None,NaN,NaN
Actividad,2023.000000,2022.000000
Rotación de Inv.,3.135348,1.900585
Rotación de A.T.,0.088641,0.077856
Rotación de C. por c.,0.000000,0.000000
None,NaN,NaN
Rentabilidad,2023.000000,2022.000000


In [5]:
r.extraccion_de_tablas('../data/raw/Ratios_empresas.xlsx',3,4,'N',22,'P', 'ratios')

Nombre de dataframe: REPSOL ratios


,2023,2022
RDT,0.720474,0.701129
RDC,2.577485,2.345922
RDLP,1.418739,1.243674
None,NaN,NaN
Actividad,2023.000000,2022.000000
Rotación de Inv.,2.049054,1.776171
Rotación de A.T.,-0.000056,0.005866
Rotación de C. por c.,0.000000,1.000000
None,NaN,NaN
Rentabilidad,2023.000000,2022.000000


In [6]:
r.extraccion_de_tablas('../data/raw/Ratios_empresas.xlsx',4,5,'P',23,'R', 'ratios')

Nombre de dataframe: REPSOL ratios


,2023,2022
RDT,0.597411,0.623825
RDC,2.349956,1.658334
RDLP,1.743271,1.623636
None,NaN,NaN
Actividad,2023.000000,2022.000000
Rotación de Inv.,-2.268268,1.541112
Rotación de A.T.,0.076193,0.090134
Rotación de C. por c.,0.000000,0.000000
None,NaN,NaN
Rentabilidad,2023.000000,2022.000000


In [7]:
r.extraccion_de_tablas('../data/raw/Ratios_empresas.xlsx',5,5,'R',23,'W', 'ratios')

Nombre de dataframe: REPSOL ratios


,2024,2023,2022,2021,2020
RDT,0.493590,0.502634,2.467448,0.742215,0.782682
RDC,0.974684,1.010592,7.679938,2.879194,3.601550
RDLP,0.392172,0.229026,1.493898,1.410693,2.593811
None,NaN,NaN,NaN,NaN,NaN
Actividad,2024.000000,2023.000000,2022.000000,2021.000000,2020.000000
Rotación de Inv.,3.459721,2.122742,0.187096,3.102503,3.308800
Rotación de A.T.,0.176963,0.186027,0.925727,0.172071,0.207441
Rotación de C. por c.,0.000000,0.000000,0.000000,0.000000,0.000000
None,NaN,NaN,NaN,NaN,NaN
Rentabilidad,2024.000000,2023.000000,2022.000000,2021.000000,2020.000000


In [8]:
r.extraccion_de_tablas('../data/raw/Ratios_empresas.xlsx',6,3,'K',21,'M', 'ratios')

Nombre de dataframe: REPSOL ratios


,2023,2022
RDT,0.747810,0.784116
RDC,2.965266,3.632110
RDLP,0.000000,NaN
None,NaN,NaN
Actividad,2023.000000,2022.000000
Rotación de Inv.,8.184344,8.507117
Rotación de A.T.,1.495057,1.638080
Rotación de C. por c.,0.000000,1.000000
None,NaN,NaN
Rentabilidad,2023.000000,2022.000000


In [3]:
r.extraccion_de_tablas('../data/raw/Ratios_empresas.xlsx',7,4,'O',22,'S', 'ratios')


Nombre de dataframe: REPSOL ratios


,2024,2023,2022,2021
RDT,0.540963,0.528463,0.589858,0.734417
RDC,1.178472,1.120723,1.438182,2.765360
RDLP,0.131222,0.124065,0.128248,1.180216
None,NaN,NaN,NaN,NaN
Actividad,2024.000000,2023.000000,2022.000000,2021.000000
Rotación de Inv.,0.767083,0.532402,0.543153,0.509664
Rotación de A.T.,0.172159,0.157379,0.214322,0.180773
Rotación de C. por c.,0.000000,0.000000,0.000000,0.000000
None,NaN,NaN,NaN,NaN
Rentabilidad,2024.000000,2023.000000,2022.000000,2021.000000


In [9]:
r.extraccion_de_tablas('../data/raw/Ratios_empresas.xlsx',8,2,'J',20,'L', 'ratios')

Nombre de dataframe: REPSOL ratios


,2023,2022
RDT,0.528337,0.566857
RDC,1.120158,1.308705
RDLP,0.356037,0.454048
None,NaN,NaN
Actividad,2023.000000,2022.000000
Rotación de Inv.,6.475615,7.703003
Rotación de A.T.,0.648347,0.803510
Rotación de C. por c.,0.000000,0.000000
None,NaN,NaN
Rentabilidad,2023.000000,2022.000000
